In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(googlesheets)
}))

gs_auth(token = "/home/users/ytanigaw/.googlesheets_token.rds")


Auto-refreshing stale OAuth token.



In [2]:
# input
header_f <- '../filtration/header.finemapping.txt'
traits_f <- '../../common/canonical_trait_names.txt'
filter_f <- '../filtration/%s_subset.tsv'
ST_table_g <- 'https://docs.google.com/spreadsheets/d/1CVm_ozbTQeAwvN6KWnn4gHuGwUNIn_O_9F7I1STg_uc'


In [3]:
header_f %>% fread() %>% colnames() -> colnames

traits_f %>% fread() %>% filter(annotation != 'Fasting_glucose') -> traits_df

traits_df %>% pull(annotation) %>%
lapply(function(trait){
    sprintf(filter_f, trait) %>% fread(col.names = colnames) %>% mutate(trait = trait)
}) %>% bind_rows() -> finemap_subset_df 


In [6]:
suppressWarnings(suppressMessages({
    c('ST6', 'ST7') %>% lapply(function(wsname){
        ST_table_g %>% gs_url() %>% gs_read(
            ws = wsname, skip=1, col_types = cols(
                'CHROM' = col_integer(), 'POS' = col_integer()
            )
        ) %>% mutate(ws_name = wsname)        
    }) %>% bind_rows() -> coding_hits_df
}))


In [8]:
coding_hits_df %>% count(ws_name)

ws_name,n
<chr>,<int>
ST6,58
ST7,1323


In [11]:
coding_hits_df %>% 
inner_join(
    finemap_subset_df,
    by=c('CHROM'='chromosome', 'POS'='position', 'REF'='allele1', 'ALT'='allele2')
) -> coding_hits_fm_df


In [14]:
coding_hits_fm_df %>% dim()


[1]  4 40

In [20]:
coding_hits_fm_df %>%
select(CHROM, POS, REF, ALT, ID, Trait, BETA, SE, prob, log10bf, ws_name)

CHROM,POS,REF,ALT,ID,Trait,BETA,SE,prob,log10bf,ws_name
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,205744218,C,T,Affx-35292504,Alkaline phosphatase,0.0218,0.0036,0,-Inf,ST7
1,205744218,C,T,Affx-35292504,Alkaline phosphatase,0.0218,0.0036,0,-Inf,ST7
1,205744218,C,T,Affx-35292504,Alkaline phosphatase,0.0218,0.0036,0,-Inf,ST7
1,205744218,C,T,Affx-35292504,Alkaline phosphatase,0.0218,0.0036,0,-Inf,ST7
